## Analysis of top 100 Billboard Songs via Spotify API
- Feature Analysis & Engineering
- Dynamic/Interactive Visualization
- Data Modeling

![alt text](https://upload.wikimedia.org/wikipedia/commons/9/95/Billboard_logo.svg) ![alt text](https://upload.wikimedia.org/wikipedia/commons/2/26/Spotify_logo_with_text.svg) 

### Recieve token --

In [1]:
import subprocess
import sys

def get_token()-> str:

    command = f'curl -X POST "https://accounts.spotify.com/api/token" \
          -H "Content-Type: application/x-www-form-urlencoded" \
          -d "grant_type=client_credentials&client_id=c92544725ea24f009b2abc7fa2c38bdb&client_secret=2a14acc2799947f2be63138957c1b31f"'  # the terminal command you want to run

    output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)

    length = None
    while length != 116:
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        code = str(output[333:-42])[3:]
        token = code.replace("'", '')
        length = len(token)
    return token

In [ ]:
token = get_token()
token

### ARTISTS ANALYSIS - (SAVE FOR LATER) (SKIP)

In [290]:
# import requests
# import csv

# # Spotify API endpoint for Tame Impala
# url = 'https://api.spotify.com/v1/artists/19Fi1Rj7kk8kyiwxpXy3yM'

# # Spotify authentication token
# token = f'Bearer  {token}'
# # Set up headers with authorization token
# headers = {
#     'Authorization': token
# }

# # Send GET request to API endpoint with headers
# response = requests.get(url, headers=headers)

# # Check if response was successful
# if response.status_code == 200:
#     # Load response data into dictionary
#     data_dict = response.json()

#     # Extract relevant data from dictionary
#     name = data_dict['name']
#     followers = data_dict['followers']['total']
#     popularity = data_dict['popularity']
#     genres = ', '.join(data_dict['genres'])

#     # Write data to CSV file
#     with open('tame_impala.csv', 'w', newline='') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerow(['Name', 'Followers', 'Popularity', 'Genres'])
#         writer.writerow([name, followers, popularity, genres])

#     print('Data saved to CSV file.')
# else:
#     print('Failed to retrieve data from API.')


Data saved to CSV file.


In [287]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# client_id = "c92544725ea24f009b2abc7fa2c38bdb"
# client_secret = "2a14acc2799947f2be63138957c1b31f"

# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# artist_name = "Chris Brown"
# results = spotify.search(q=artist_name, type='artist')
# artist_id = results['artists']['items'][0]['id']

In [292]:
## Do something here with top 100 artists --> 

### Songs
- SCRAPE TOP 100 SONGS + ARTISTS from BILLBOARD 100: https://www.billboard.com/charts/hot-100/

In [1]:
from bs4 import BeautifulSoup
import requests
response = requests.get('https://www.billboard.com/charts/hot-100/')
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')
#soup.prettify()
chart_rows = soup.find_all('li')
li = soup.find_all('li')
#lis = soup.find_all('li', {'class': 'o-chart-results-list__item // lrv-u-flex-grow-1 lrv-u-flex lrv-u-flex-direction-column lrv-u-justify-content-center lrv-u-border-b-1 u-border-b-0@mobile-max lrv-u-border-color-grey-light lrv-u-padding-l-1@mobile-max'})


In [5]:
h3s = [h3 for l in li for h3 in l.find_all('h3', {'id' : "title-of-a-story"})]
top100 = []
cnt=0
for h3 in h3s:
    cnt+=1
    top100.append(h3.text.strip())
top100 = top100[::2]

In [6]:
#Scrape top 100 Artists
artists_ls = []
for val in chart_rows:
    # Find the span element with the given class and print its text
    span_elem = val.find('span', {'class': "c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet"})
    if span_elem:
        artists_ls.append(span_elem.text.strip())
        
for val in chart_rows:
    # Find the span element with the given class and print its text
    span_elem = val.find('span', {'class': "c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only"})
    if span_elem:
        artists_ls.append(span_elem.text.strip())
        
artists_ls = artists_ls[::2]

In [7]:
len(top100), len(artists_ls)

(100, 100)

In [9]:
! pip install spotipy

In [8]:
#GET SPOTIFY ID FOR SONGS TO FEED API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = "31c8b9cbb5c14ae2912c988dfdf59d82"
client_secret = "f62b5c5c27644a088e27a3f30034e65c"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

track_ids = []

for idx in range(len(top100)):
    query = top100[idx]
    results = spotify.search(q=query, type='track')

    track_id = results['tracks']['items'][0]['id']
    track_ids.append(track_id)

In [9]:
len(track_ids)

100

In [2]:
track_ids.head()

NameError: name 'track_ids' is not defined

### CONNECT TO API & QUERY RESULTS

In [10]:
import requests
import csv

dnc_ls, energy_ls, loud_ls, live_ls, tempo_ls = [], [], [], [], []
# Spotify API endpoint for Tame Impala
for idx in range(len(track_ids)):

    url = f'https://api.spotify.com/v1/audio-features/{track_ids[idx]}'

    # Spotify authentication token
    token = f'Bearer  {get_token()}'

    headers = {'Authorization': token}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data_dict = response.json()
        dnc_ls.append(data_dict['danceability'])
        energy_ls.append(data_dict['energy'])
        loud_ls.append(data_dict['loudness'])
        live_ls.append(data_dict['liveness'])
        tempo_ls.append(data_dict['tempo'])                                 

In [11]:
# Write data to CSV file
with open('top100.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Artist', 'Song', 'Danceability', 'energy', 'loudness', 'liveness', 'tempo'])
    for idx in range(len(top100)):        
        writer.writerow([artists_ls[idx], top100[idx], dnc_ls[idx], energy_ls[idx], 
                         loud_ls[idx],live_ls[idx], tempo_ls[idx]])

In [13]:
import pandas as pd
songs_df = pd.DataFrame({'Artist': artists_ls, 'Song': top100, 'Danceability': dnc_ls,
                        'Energy': energy_ls, 'Loudness': loud_ls, 'Livenss': live_ls, 'Tempo': tempo_ls})

songs_df.head(5)

,Artist,Song,Danceability,Energy,Loudness,Livenss,Tempo
0,Morgan Wallen,Last Night,0.517,0.675,-5.382,0.1510,203.853
1,SZA,Kill Bill,0.644,0.735,-5.747,0.1610,88.980
2,Miley Cyrus,Flowers,0.707,0.681,-4.325,0.0322,117.999
3,Eslabon Armado X Peso Pluma,Ella Baila Sola,0.668,0.758,-5.176,0.0837,147.989
4,Rema & Selena Gomez,Calm Down,0.801,0.806,-5.206,0.1140,106.999


## convert to csv

In [14]:
# save DataFrame as CSV file
songs_df.to_csv('songs_data.csv', index=False)
print("CSV file saved successfully!")

CSV file saved successfully!
